In [ ]:
import pandas as pd
import polars as pl
import time
from content.utils import polars_read_tsv_file

DATASET_FILE = "content/data/imdb_dataset/{file}"

def read_imdb_tsv_file(filename_stem):
    return polars_read_tsv_file(DATASET_FILE + f"{filename_stem}")

def scan_imdb_tsv_file(filename_stem):
    return pl.scan_csv(DATASET_FILE + f"{filename_stem}")

def read_imdb_parquet_file(filename_stem):
    return pl.read_parquet(DATASET_FILE + f"{filename_stem}.parquet")

def scan_imdb_parquet_file(filename_stem):
    return pl.scan_parquet(DATASET_FILE + f"{filename_stem}.parquet")

## Reading CSV in Polars

In Polars, files can be read directly into memory just as in Pandas.
Users of Pandas might know that there is an option to provide a selection of columns you want to read to save resources.
Polars, however, takes this a bit further.
In Polars you have a lazy loading option, which permits scanning files instead of reading them directly into memory.
This creates an instance of a `LazyFrame` instead of a `DataFrame`.
The scanning operation keeps a reference to the file, while permitting users to define any processing of the data without ever reading the file.
The file conents are then only read when the user finally collects the data by calling `LazyFrame.collect()`.
As a consequence, Polars can reorder any operations performed on the `LazyFrame` to create an optimal query plan, which often allows for much faster processing with a significantly lower memory usage.

Let us read the file containing all movie titles from the IMDB dataset and inspect the first 15 titles.

In [ ]:

start_time = time.time()
movie_titles = read_imdb_tsv_file("title.basics")
print(f"{time.time() - start_time} seconds for Polars")
display(movie_titles.head(15))

As you can see, this takes quite a bit of time.
What if we only scan the file, filter the first 15 items and only then collect the data?

In [ ]:
start_time = time.time()
movie_titles = scan_imdb_tsv_file("title.basics")
print(f"{time.time() - start_time} seconds for Polars")
display(movie_titles.head(15).collect())
del movie_titles

Ah, much better!
The scanning operation and the consequent slicing operation to select the first 15 items allows Polars to only read partially read the file and return the 15 rows as soon as they have been read.

What would Pandas do?

In [ ]:

start_time = time.time()
movies_pandas = pd.read_csv(DATASET_FILE.format(file="title.basics.tsv"), sep="\t")
movies_pandas.head(15)
print(f"{time.time() - start_time} seconds for Pandas")

## I want to watch a good movie!

With the knowledge we obtained above, let's only scan the movies and the corresponding ratings (defined in a separate file).
To find the movies that are the best according to the users who rated, we will have to join the two together.
Polars defines joins on this with different join strategy types which most of you will know from databases theory and any packages that deal with data.
Apart from `left`, `right`, `inner`, `outer` and `cross` joins, the `semi` and `anti` join are also available in Polars.

### Exercise 4.2
Create a ranking of the 20 best movies according to the user ratings using the `LazyFrame` instance.
For this, you will have to join the ratings with the movie data.
Don't forget the number of votes, or your ranking will be worthless.
Can you get close to the official IMDB ranking?

In [ ]:
movie_titles = read_imdb_tsv_file("title.basics")
ratings = read_imdb_tsv_file("title.ratings")


### Exercise 4.3

Let's take it one step further!
Can we find the 20 actors whose known-for movies are the highest ranking movies on average?
Again, do not forget to take the number of votes into account as well!
For this you may have to `explode()` the data frame and may additionally need to `split()` a column with comma-separated values.

In [ ]:
actors = scan_imdb_tsv_file("name.basics")

As the last query is likely quite a complex one, it can provide a nice example for the query optimization of Polars.
If you did not manage to do the last one or skipped it, you can also do this with the query from before.
In that case, if the query is already optimal, you may want to reverse the order or introduce some extra (unnecessary) statements, such as additional sorting operations before the final sort.

The LazyFrame, also sometimes referred to as a query as it builds up the query through the operations the user defines on the object, provides insights in how queries are built up and how Polars automatically optimizes the query for us before collecting the data.
These insights can be obtained through the `explain` method, which gives an overview of the query steps in text, or through the `show_graph` method, which gives a visual overview of the steps through an extra extension, as shown below.
For both of these methods you can pass in `True` or `False` for the parameter `optimized` in order to compare the optimized with the naive query plan.

### Exercise 4.4
Use the `explain()` and `show_graph()` method on the `LazyFrame` instance constructed in the last exercise to compare the optimized query with the naive one.

## Parquet files

CSV (or TSV) files are easy to work with, human-readable and supported by a wide range of different tools (if you have to work with people who only want to deal with Excel...).
However, it is typically not the most efficient format for storing data.
One of the reasons is that data is stored in row-based fashion, which does not allow us to efficiently read only required columns.
Apart from that, there is no compression and intelligent rearrangement of data to avoid having to read only a certain portion of it.

This is where Parquet comes in!
Parquet is a file format which originates from an open-source project under the Apache Software Foundation (hence the full name Apache Parquet).
In contrast to CSV, the format is a columnar-storage format.
This change also opened up possibilities for column-based encoding and compression, which now allows for optimal techniques for the types contained in the column, which is stored along with the contents itself.

Let's use Polars to read in the CSV files and convert them to Parquet files!

### Exercise 4.5
#### Exercise 4.5.1
Convert the title.ratings and the name.basics files to Parquet files using Polars.

#### Exercise 4.5.2 
Would there be any speed difference if we now perform the same operations as above on the CSV files to obtain a list of the best-ranked movies?
Rewrite exercise 4.2 to use the Parquet files instead and measure the difference in execution time.

In [ ]:
movie_titles = read_imdb_parquet_file("title.basics")
ratings = read_imdb_parquet_file("title.ratings")


In [ ]:
movie_titles = scan_imdb_parquet_file("title.basics")
ratings = scan_imdb_parquet_file("title.ratings")
